# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

In [913]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

In [914]:
data = pd.read_excel("mcdonalds_tweets.xlsx",sheet_name = "Treinamento")
colunas = ["Treinamento", "Classificação"]
data = data.loc[:,colunas]


#Limpando caracteres irrelevantes:

def deletar_caracteres(coluna):
    coluna = coluna.replace("."," ").replace(":"," ").replace(","," ").replace("'","").replace('"', " ").replace\
    ("rt"," ").replace("#"," ").replace("-","")
    return coluna

    

data['Treinamento'] = data['Treinamento'].apply(deletar_caracteres)



In [915]:
#Dividindo entre relevante e irrelevante
Relevante = data[data["Classificação"] == 1]
irrel = data[data["Classificação"] == 0] 

dic_palavras_rel = {}
dic_palavras_irrel = {}

for i in Relevante["Treinamento"]:
    x = i.split()
    for e in x:
        if e not in dic_palavras_rel:
            dic_palavras_rel[e] = 1
        
        elif e in dic_palavras_pos:
            dic_palavras_rel[e] += 1



            
for i in irrel["Treinamento"]:
    x = i.split()
    for e in x:
        if e not in dic_palavras_irrel:
            dic_palavras_irrel[e] = 1
        
        elif e in dic_palavras_irrel:
            dic_palavras_irrel[e] += 1
            

dic_palavras_re = pd.DataFrame.from_dict(dic_palavras_rel, orient='index')
dic_palavras_irre =  pd.DataFrame.from_dict(dic_palavras_irrel,  orient='index')


dic_palavras_re.columns = ["Relevante"]
dic_palavras_irre.columns = ["Irrelevante"]


Dataframe =  dic_palavras_re.join(dic_palavras_irre, how='outer')


Dataframe = Dataframe.fillna(0)
Dataframe.reset_index(level=0, inplace=True)
Dataframe.rename(columns = {"index":'Palavras'}, inplace=True)

Dataframe['Relevante'] = Dataframe['Relevante'].astype(int)
Dataframe['Irrelevante'] = Dataframe['Irrelevante'].astype(int)
Dataframe.head(10)






,Palavras,Relevante,Irrelevante
0,&amp;,0,1
1,&gt;,0,8
2,(@,2,0
3,(sabado,1,0
4,//t,37,65
5,000,1,0
6,01,1,0
7,1,1,0
8,10,1,1
9,123m,0,1


In [916]:
# Total de palavras positivas, negativas e irrelevantes

quantidade_rel = 0

quantidade_irrel = 0  


for e in Dataframe.Relevante:
    quantidade_rel +=e
    
    
        
for f in Dataframe.Irrelevante:
        quantidade_irrel +=f 
        
print(quantidade_rel)    

print(quantidade_irrel)  


2448
2012


In [917]:
#Probabilidade de palavra/relevante e palavra/irrelevante
dic2 = {}

dic0 = {}
lista2 = []

lista0 = []

for e in Dataframe.Relevante:
    lista2.append(((e+1)/(quantidade_rel+ total_palavras)))
  
    
for e in Dataframe.Irrelevante:
    lista0.append(((e+1)/(quantidade_irrel+ total_palavras)))
    
    
for i in range(0,len(Dataframe.Relevante)):
    dic2[Dataframe.Palavras[i]] = lista2[i]
    

    
for i in range(0,len(Dataframe.Irrelevante)):
    dic0[Dataframe.Palavras[i]] = lista0[i]
        
index = pd.RangeIndex(start=0, stop=1, step=1)  

    
dc2 = pd.DataFrame(data=dic2,index=index)
x=dc2.T
dc0 = pd.DataFrame(data=dic0,index=index)
z=dc0.T
x.columns = ["Palavra_Relevante"]
z.columns = ["Palavra_Irrelevante"]

Dataframe2 =  x.join(z, how='inner')
Dataframe2.head(10)


,Palavra_Relevante,Palavra_Irrelevante
&amp;,0.000145,0.000309
&gt;,0.000145,0.001391
(@,0.000434,0.000155
(sabado,0.000290,0.000155
//t,0.005501,0.010198
000,0.000290,0.000155
01,0.000290,0.000155
1,0.000290,0.000155
10,0.000290,0.000309
123m,0.000145,0.000309


In [918]:
# Probabilidade do tweet ser relevante ou irrelevante

total_1 = data['Treinamento'][data['Classificação'] == 1].count()
total_2 = data['Treinamento'][data['Classificação'] == 0].count()

PRelevante = total_1/(total_1+total_2)
Pirrelevante = total_2/(total_1+total_2)

print(PRelevante)
print(Pirrelevante)




0.59
0.41


In [919]:
lis1 = []
lis2 = []
for e in Dataframe2.Palavra_Relevante:
    lis1.append(e*PRelevante)
for e in Dataframe2.Palavra_Irrelevante:
    lis2.append(e*Pirrelevante)


    
Prob = {}    

Prob2 = {}
        
        
for i in range(0,len(Dataframe.Relevante)):
    Prob[Dataframe.Palavras[i]] = lis1[i]
    

    
for i in range(0,len(Dataframe.Irrelevante)):
    Prob2[Dataframe.Palavras[i]] = lis2[i]
        
index = pd.RangeIndex(start=0, stop=1, step=1)  

    
dc = pd.DataFrame(data=Prob,index=index)
x=dc.T
dc0 = pd.DataFrame(data=Prob2,index=index)
z=dc0.T
x.columns = ["Relevante_Palavra"]
z.columns = ["Irrelevante_Palavra"]

Dataframe3 =  x.join(z, how='inner')
Dataframe3.head(20)
        

        
            


 


,Relevante_Palavra,Irrelevante_Palavra
&amp;,0.000085,0.000127
&gt;,0.000085,0.000570
(@,0.000256,0.000063
(sabado,0.000171,0.000063
//t,0.003246,0.004181
000,0.000171,0.000063
01,0.000171,0.000063
1,0.000171,0.000063
10,0.000171,0.000127
123m,0.000085,0.000127


___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
